<a href="https://colab.research.google.com/github/csukuangfj/colab/blob/master/sherpa_onnx_streaming_paraformer_cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This colab notebook shows how to use [sherpa-onnx](https://github.com/k2-fsa/sherpa-onnx) to run streaming [Paraformer](https://www.modelscope.cn/models/damo/speech_paraformer_asr_nat-zh-cn-16k-common-vocab8404-online/summary) models on CPU.



# Install sherpa-onnx

In [1]:
%%shell

pip install sherpa-onnx

# Please refer to
# https://k2-fsa.github.io/sherpa/onnx/install/index.html
# for other installation methods

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.4 MB/s eta 0:00:00


Check that `sherpa-onnx` has been installed successfully:

In [2]:
%%shell

# C++ binaries: Decoding files with streaming models
sherpa-onnx --help

# C++ binaries: Streaming WebSocket server
sherpa-onnx-online-websocket-server --help

# Test Python binding
python3 -c "import sherpa_onnx; print(sherpa_onnx.__file__, sherpa_onnx.__version__)"


/project/sherpa-onnx/csrc/parse-options.cc:PrintUsage:402 

Usage:

  ./bin/sherpa-onnx \
    --tokens=/path/to/tokens.txt \
    --encoder=/path/to/encoder.onnx \
    --decoder=/path/to/decoder.onnx \
    --joiner=/path/to/joiner.onnx \
    --provider=cpu \
    --num-threads=2 \
    --decoding-method=greedy_search \
    /path/to/foo.wav [bar.wav foobar.wav ...]

Note: It supports decoding multiple files in batches

Default value for num_threads is 2.
Valid values for decoding_method: greedy_search (default), modified_beam_search.
Valid values for provider: cpu (default), cuda, coreml.
foo.wav should be of single channel, 16-bit PCM encoded wave file; its
sampling rate can be arbitrary and does not need to be 16kHz.

Please refer to
https://k2-fsa.github.io/sherpa/onnx/pretrained_models/index.html
for a list of pre-trained models to download.

Options:
  --max-active-paths          : beam size used in modified beam search. (int, default = 4)
  --debug                     : true to print

# Download pre-trained streaming paraformer model

Please see
https://k2-fsa.github.io/sherpa/onnx/pretrained_models/online-paraformer/index.html
for details

In [3]:
%%shell

sudo apt-get install git-lfs
git clone https://huggingface.co/csukuangfj/sherpa-onnx-streaming-paraformer-bilingual-zh-en
ls -lh sherpa-onnx-streaming-paraformer-bilingual-zh-en

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
Cloning into 'sherpa-onnx-streaming-paraformer-bilingual-zh-en'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), 949.80 KiB | 9.22 MiB/s, done.
Filtering content: 100% (4/4), 1.02 GiB | 23.34 MiB/s, done.
total 1.1G
-rw-r--r-- 1 root root  69M Aug 14 13:02 decoder.int8.onnx
-rw-r--r-- 1 root root 218M Aug 14 13:02 decoder.onnx
-rw-r--r-- 1 root root 158M Aug 14 13:02 encoder.int8.onnx
-rw-r--r-- 1 root root 607M Aug 14 13:03 encoder.onnx
-rw-r--r-- 1 root root  415 Aug 14 13:02 README.md
drwxr-xr-x 2 root root 4.0K Aug 14 13:02 test_wavs
-rw-r--r-- 1 root root  74K Aug 14 13:02 tokens.txt


# Real-time factor (RTF) test

## float32 models

In [4]:
%%shell

sherpa-onnx \
  --tokens=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt \
  --paraformer-encoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.onnx \
  --paraformer-decoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.onnx \
  --num-threads=1 \
  ./sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav

/project/sherpa-onnx/csrc/parse-options.cc:Read:361 sherpa-onnx --tokens=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt --paraformer-encoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.onnx --paraformer-decoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.onnx --num-threads=1 ./sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav 

OnlineRecognizerConfig(feat_config=FeatureExtractorConfig(sampling_rate=16000, feature_dim=80), model_config=OnlineModelConfig(transducer=OnlineTransducerModelConfig(encoder="", decoder="", joiner=""), paraformer=OnlineParaformerModelConfig(encoder="./sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.onnx", decoder="./sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.onnx"), tokens="./sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt", num_threads=1, debug=False, provider="cpu", model_type=""), lm_config=OnlineLMConfig(model="", scale=0.5), endpoint_config=EndpointConfig(rule1=En

## int8 models

In [5]:
%%shell

sherpa-onnx \
  --tokens=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt \
  --paraformer-encoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.int8.onnx \
  --paraformer-decoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.int8.onnx \
  --num-threads=1 \
  ./sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav

/project/sherpa-onnx/csrc/parse-options.cc:Read:361 sherpa-onnx --tokens=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt --paraformer-encoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.int8.onnx --paraformer-decoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.int8.onnx --num-threads=1 ./sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav 

OnlineRecognizerConfig(feat_config=FeatureExtractorConfig(sampling_rate=16000, feature_dim=80), model_config=OnlineModelConfig(transducer=OnlineTransducerModelConfig(encoder="", decoder="", joiner=""), paraformer=OnlineParaformerModelConfig(encoder="./sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.int8.onnx", decoder="./sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.int8.onnx"), tokens="./sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt", num_threads=1, debug=False, provider="cpu", model_type=""), lm_config=OnlineLMConfig(model="", scale=0.5), endpoint_config=End

# Start the C++ streaming websocket server

In [6]:
! nproc

2


This colab notebook provides only 2 CPUs.

In [7]:
%%shell

nohup sherpa-onnx-online-websocket-server \
  --tokens=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt \
  --paraformer-encoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.int8.onnx \
  --paraformer-decoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.int8.onnx \
  --port=6006 \
  --num-work-threads=1 \
  --num-io-threads=1 > cpp-log-server.txt &

nohup: redirecting stderr to stdout


In [8]:
%%shell
# sleep 10 seconds to wait for the server startup
sleep 10
cat cpp-log-server.txt

/project/sherpa-onnx/csrc/parse-options.cc:Read:361 sherpa-onnx-online-websocket-server --tokens=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt --paraformer-encoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.int8.onnx --paraformer-decoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.int8.onnx --port=6006 --num-work-threads=1 --num-io-threads=1 

/project/sherpa-onnx/csrc/online-websocket-server.cc:main:79 Started!
/project/sherpa-onnx/csrc/online-websocket-server.cc:main:80 Listening on: 6006
/project/sherpa-onnx/csrc/online-websocket-server.cc:main:81 Number of work threads: 1


## Start a C++ webscoket client to decode files

In [9]:
%%shell

sherpa-onnx-online-websocket-client \
  --server-ip=127.0.0.1  \
  --server-port=6006 \
  sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav

/project/sherpa-onnx/csrc/parse-options.cc:Read:361 sherpa-onnx-online-websocket-client --server-ip=127.0.0.1 --server-port=6006 sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav 

/project/sherpa-onnx/csrc/online-websocket-client.cc:SendMessage:114 Starting to send audio
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMessage:91 {"is_final":false,"segment":0,"start_time":0.0,"text":"","timestamps":"[]","tokens":[]}
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMessage:91 {"is_final":false,"segment":0,"start_time":0.0,"text":"昨","timestamps":"[]","tokens":["昨"]}
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMessage:91 {"is_final":false,"segment":0,"start_time":0.0,"text":"昨天是","timestamps":"[]","tokens":["昨","天","是"]}
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMessage:91 {"is_final":false,"segment":0,"start_time":0.0,"text":"昨天是 mon","timestamps":"[]","tokens":["昨","天","是","mon@@"]}
/project/sherpa-onnx/csrc/online-websocket-client.cc

## Start a Python webscoket client to decode files

In [10]:
%%shell
# Download the python client code first
# https://github.com/k2-fsa/sherpa-onnx/blob/master/python-api-examples/online-websocket-client-decode-file.py

wget https://raw.githubusercontent.com/k2-fsa/sherpa-onnx/master/python-api-examples/online-websocket-client-decode-file.py

--2023-08-14 13:04:04--  https://raw.githubusercontent.com/k2-fsa/sherpa-onnx/master/python-api-examples/online-websocket-client-decode-file.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4797 (4.7K) [text/plain]
Saving to: ‘online-websocket-client-decode-file.py’

online-websocket-cl 100%[===================>]   4.68K  --.-KB/s    in 0s      

2023-08-14 13:04:05 (37.5 MB/s) - ‘online-websocket-client-decode-file.py’ saved [4797/4797]



In [11]:
! pip install websockets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 2.7 MB/s eta 0:00:00


In [12]:
%%shell

python3 ./online-websocket-client-decode-file.py \
  --server-addr localhost \
  --server-port 6006 \
  sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav

2023-08-14 13:04:12,288 INFO [online-websocket-client-decode-file.py:159] {'server_addr': 'localhost', 'server_port': 6006, 'samples_per_message': 8000, 'seconds_per_message': 0.1, 'sound_file': 'sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav'}
2023-08-14 13:04:12,295 INFO [online-websocket-client-decode-file.py:133] Sending sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav
2023-08-14 13:04:12,524 INFO [online-websocket-client-decode-file.py:115] {"is_final":false,"segment":0,"start_time":0.0,"text":"","timestamps":"[]","tokens":[]}
2023-08-14 13:04:12,675 INFO [online-websocket-client-decode-file.py:115] {"is_final":false,"segment":0,"start_time":0.0,"text":"昨","timestamps":"[]","tokens":["昨"]}
2023-08-14 13:04:12,832 INFO [online-websocket-client-decode-file.py:115] {"is_final":false,"segment":0,"start_time":0.0,"text":"昨天是","timestamps":"[]","tokens":["昨","天","是"]}
2023-08-14 13:04:12,999 INFO [online-websocket-client-decode-file.py:115] {"is_final":

Note that the C++ streaming WebSocket is able to handle multiple client connecting at the same time.

# Start the Python C++ streaming websocket server

The server code is contained in the following folder:

https://github.com/k2-fsa/sherpa-onnx/tree/master/python-api-examples

It consits of:
 - [streaming_server.py](https://github.com/k2-fsa/sherpa-onnx/blob/master/python-api-examples/streaming_server.py); it contains a WebSocker server and a simple HTTP server. The two servers share the same port

 - [web](https://github.com/k2-fsa/sherpa-onnx/tree/master/python-api-examples/web); it contains HTML files so that users can use a browser to interact with the server.

Instead of using `wget`, we use `git clone` below to download the code:

In [13]:
%%shell

git clone https://github.com/k2-fsa/sherpa-onnx

Cloning into 'sherpa-onnx'...
remote: Enumerating objects: 2855, done.
remote: Counting objects: 100% (1332/1332), done.
remote: Compressing objects: 100% (532/532), done.
remote: Total 2855 (delta 915), reused 1006 (delta 780), pack-reused 1523
Receiving objects: 100% (2855/2855), 2.02 MiB | 13.79 MiB/s, done.
Resolving deltas: 100% (1647/1647), done.


Now we can start the Python streaming websocket server:

In [14]:
%%shell

nohup python3 ./sherpa-onnx/python-api-examples/streaming_server.py \
  --tokens=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt \
  --paraformer-encoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.int8.onnx \
  --paraformer-decoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.int8.onnx \
  --port=6008 \
  --doc-root=./sherpa-onnx/python-api-examples/web > python-server-log.txt &

# sleep 10 seconds to wait for the server startup
sleep 10

nohup: redirecting stderr to stdout


In [15]:
%%shell

cat ./python-server-log.txt

2023-08-14 13:04:16,659 INFO [streaming_server.py:719] {'encoder': None, 'decoder': None, 'joiner': None, 'paraformer_encoder': './sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.int8.onnx', 'paraformer_decoder': './sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.int8.onnx', 'tokens': './sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt', 'sample_rate': 16000, 'feat_dim': 80, 'provider': 'cpu', 'decoding_method': 'greedy_search', 'num_active_paths': 4, 'use_endpoint': 1, 'rule1_min_trailing_silence': 2.4, 'rule2_min_trailing_silence': 1.2, 'rule3_min_utterance_length': 20, 'port': 6008, 'nn_pool_size': 1, 'max_batch_size': 50, 'max_wait_ms': 10, 'max_message_size': 1048576, 'max_queue_size': 32, 'max_active_connections': 500, 'num_threads': 2, 'certificate': None, 'doc_root': './sherpa-onnx/python-api-examples/web'}
2023-08-14 13:04:21,357 INFO [streaming_server.py:547] No certificate provided
2023-08-14 13:04:21,359 INFO [server.py:711] server listening on 

Now you can visit the address
  http://localhost:6008
on your browser.

Or you can use a websocket client to interact with the server:

## Start a C++ webscoket client to decode files

In [16]:
%%shell

sherpa-onnx-online-websocket-client \
  --server-ip=127.0.0.1  \
  --server-port=6008 \
  sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav

/project/sherpa-onnx/csrc/parse-options.cc:Read:361 sherpa-onnx-online-websocket-client --server-ip=127.0.0.1 --server-port=6008 sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav 

/project/sherpa-onnx/csrc/online-websocket-client.cc:SendMessage:114 Starting to send audio
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMessage:91 {"text": "", "segment": 0}
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMessage:91 {"text": "\u6628", "segment": 0}
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMessage:91 {"text": "\u6628\u5929\u662f", "segment": 0}
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMessage:91 {"text": "\u6628\u5929\u662f mon", "segment": 0}
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMessage:91 {"text": "\u6628\u5929\u662f monday", "segment": 0}
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMessage:91 {"text": "\u6628\u5929\u662f monday", "segment": 0}
/project/sherpa-onnx/csrc/online-websocket-client.cc:OnMes

In [17]:
print('\u6628\u5929\u662f monday today day is \u96f6\u516b\u4e8c the day after tomorrow \u662f\u661f\u671f\u4e09')

昨天是 monday today day is 零八二 the day after tomorrow 是星期三


## Start a Python webscoket client to decode files

In [18]:
%%shell

# Note that the python server is listening on port 6008

python3 ./online-websocket-client-decode-file.py \
  --server-addr localhost \
  --server-port 6008 \
  sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav

2023-08-14 13:04:31,329 INFO [online-websocket-client-decode-file.py:159] {'server_addr': 'localhost', 'server_port': 6008, 'samples_per_message': 8000, 'seconds_per_message': 0.1, 'sound_file': 'sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav'}
2023-08-14 13:04:31,339 INFO [online-websocket-client-decode-file.py:133] Sending sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav
2023-08-14 13:04:31,599 INFO [online-websocket-client-decode-file.py:115] {"text": "", "segment": 0}
2023-08-14 13:04:31,808 INFO [online-websocket-client-decode-file.py:115] {"text": "\u6628", "segment": 0}
2023-08-14 13:04:32,027 INFO [online-websocket-client-decode-file.py:115] {"text": "\u6628\u5929\u662f", "segment": 0}
2023-08-14 13:04:32,278 INFO [online-websocket-client-decode-file.py:115] {"text": "\u6628\u5929\u662f mon", "segment": 0}
2023-08-14 13:04:32,879 INFO [online-websocket-client-decode-file.py:115] {"text": "\u6628\u5929\u662f monday", "segment": 0}
2023-08-14 13:

In [19]:
print('\u6628\u5929\u662f monday today day is \u96f6\u516b\u4e8c the day after tomorrow \u662f\u661f\u671f\u4e09')

昨天是 monday today day is 零八二 the day after tomorrow 是星期三


# Speech recognition from microphones

## Websocket client with a microphone

In [20]:
%%shell

sudo apt-get install libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0
pip install sounddevice

python3 ./sherpa-onnx/python-api-examples/online-websocket-client-microphone.py --help

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libasound2-dev' instead of 'libasound-dev'
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (470 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be us

## Standalone python script with a microphone

### Without endpoint detection

In [21]:
%%shell

python3 ./sherpa-onnx/python-api-examples/speech-recognition-from-microphone.py --help


usage: speech-recognition-from-microphone.py
       [-h]
       --tokens
       TOKENS
       --encoder
       ENCODER
       --decoder
       DECODER
       [--joiner JOINER]
       [--decoding-method DECODING_METHOD]
       [--max-active-paths MAX_ACTIVE_PATHS]
       [--provider PROVIDER]
       [--bpe-model BPE_MODEL]
       [--modeling-unit MODELING_UNIT]
       [--contexts CONTEXTS]
       [--context-score CONTEXT_SCORE]

options:
  -h, --help
    show this
    help
    message and
    exit
  --tokens TOKENS
    Path to
    tokens.txt
    (default:
    None)
  --encoder ENCODER
    Path to the
    encoder
    model
    (default:
    None)
  --decoder DECODER
    Path to the
    decoder
    model
    (default:
    None)
  --joiner JOINER
    Path to the
    joiner
    model
    (default:
    None)
  --decoding-method DECODING_METHOD
    Valid
    values are 
    greedy_sear
    ch and modi
    fied_beam_s
    earch
    (default: g
    reedy_searc
    h)
  --max-active-paths MAX_AC

### With endpoint detection

In [22]:
%%shell

python3 ./sherpa-onnx/python-api-examples/speech-recognition-from-microphone-with-endpoint-detection.py --help

usage: speech-recognition-from-microphone-with-endpoint-detection.py
       [-h]
       --tokens
       TOKENS
       --encoder
       ENCODER
       --decoder
       DECODER
       --joiner
       JOINER
       [--decoding-method DECODING_METHOD]
       [--provider PROVIDER]

options:
  -h, --help
    show this
    help
    message and
    exit
  --tokens TOKENS
    Path to
    tokens.txt
    (default:
    None)
  --encoder ENCODER
    Path to the
    encoder
    model
    (default:
    None)
  --decoder DECODER
    Path to the
    decoder
    model
    (default:
    None)
  --joiner JOINER
    Path to the
    joiner
    model
    (default:
    None)
  --decoding-method DECODING_METHOD
    Valid
    values are 
    greedy_sear
    ch and modi
    fied_beam_s
    earch
    (default: g
    reedy_searc
    h)
  --provider PROVIDER
    Valid
    values:
    cpu, cuda,
    coreml
    (default:
    cpu)


# Decode files with a standalone Python script

In [23]:
%%shell
python3 ./sherpa-onnx/python-api-examples/online-decode-files.py \
  --tokens=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/tokens.txt \
  --paraformer-encoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/encoder.int8.onnx \
  --paraformer-decoder=./sherpa-onnx-streaming-paraformer-bilingual-zh-en/decoder.int8.onnx \
  ./sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav

Started!
Done!
./sherpa-onnx-streaming-paraformer-bilingual-zh-en/test_wavs/0.wav
昨天是 monday today day is 零八二 the day after tomorrow 是星期三
----------
num_threads: 1
decoding_method: greedy_search
Wave duration: 10.053 s
Elapsed time: 4.420 s
Real time factor (RTF): 4.420/10.053 = 0.440


# What's Next

## Try streaming Paraformer with Android

Please refer to
https://k2-fsa.github.io/sherpa/onnx/android/index.html


## Try streaming Paraformer with iOS

Please refer to
https://k2-fsa.github.io/sherpa/onnx/ios/index.html

## Try streaming Paraformer with 64-bit ARM

For instance, you can run streaming paraformer with
Rapsberry Pi.

Please see
- Cross compiling: https://k2-fsa.github.io/sherpa/onnx/install/aarch64-embedded-linux.html
- Direct compiling: https://k2-fsa.github.io/sherpa/onnx/install/linux.html


## Try streaming Paraformer with 32-bit ARM

Please see

- Cross compiling: https://k2-fsa.github.io/sherpa/onnx/install/arm-embedded-linux.html
- Direct compiling: https://k2-fsa.github.io/sherpa/onnx/install/linux.html


## Try streaming Paraformer on Windows with MFC

- Please download the `exe` from https://github.com/k2-fsa/sherpa-onnx/releases/download/v1.7.5/sherpa-onnx-streaming-v1.7.5.exe

## Try streaming Paraformer with Go API

Please see
- https://k2-fsa.github.io/sherpa/onnx/go-api/index.html
- https://github.com/k2-fsa/sherpa-onnx/tree/master/go-api-examples

## Try streaming Paraformer with `C#` API

Please see
- https://k2-fsa.github.io/sherpa/onnx/csharp-api/index.html
- https://github.com/k2-fsa/sherpa-onnx/tree/master/dotnet-examples

## Try streaming Paraformer with C API

Please see
- https://k2-fsa.github.io/sherpa/onnx/c-api/index.html
- https://github.com/k2-fsa/sherpa-onnx/tree/master/c-api-examples

## Try streaming Paraformer on GPU

Please see

https://github.com/k2-fsa/colab/blob/master/sherpa-onnx/sherpa_onnx_streaming_paraformer_gpu.ipynb
